# ETL

## Extract

script to extract data from csv, json and parquet files

In [1]:
from Extract import extract

folder = 'data/'
data = extract(folder=folder)

loaded ids_0.csv of shape : (1001, 79)
loaded ids_1.csv of shape : (1001, 79)
loaded ids_2.csv of shape : (1001, 79)
loaded ids_3.json of shape : (1001, 79)
loaded ids_4.json of shape : (1001, 79)
loaded ids_5.parquet of shape : (15001, 79)
loaded ids_6.parquet of shape : (5001, 79)
loaded ids_7.json of shape : (9000, 79)
loaded ids_8.parquet of shape : (10293, 79)
loaded ids_9.json of shape : (10293, 79)


saving data into completeData.csv to prevent repeat extraction

In [2]:
# saving the df as a file so we dont need to repeatedly perform the extraction step
#data.to_csv('completeData.csv', index=False) 
#data = pd.read_csv('completeData.csv')

## Transform

In [3]:
#displaying the shape of our data as well as the first rows
data.shape , data.head()

((54593, 79),
     Destination Port   Flow Duration   Total Fwd Packets  \
 0              36102              23                   2   
 1                 53           30805                   2   
 2                443         5535509                   8   
 3                 53           49235                   4   
 4                 53             181                   2   
 
     Total Backward Packets  Total Length of Fwd Packets  \
 0                        1                           31   
 1                        2                          108   
 2                        8                          372   
 3                        2                          152   
 4                        2                           70   
 
     Total Length of Bwd Packets   Fwd Packet Length Max  \
 0                             6                      31   
 1                           230                      54   
 2                          3876                     191   
 3              

In [4]:
# display datas type and non empty values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54593 entries, 0 to 54592
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0    Destination Port             54593 non-null  int64  
 1    Flow Duration                54593 non-null  int64  
 2    Total Fwd Packets            54593 non-null  int64  
 3    Total Backward Packets       54593 non-null  int64  
 4   Total Length of Fwd Packets   54593 non-null  int64  
 5    Total Length of Bwd Packets  54593 non-null  int64  
 6    Fwd Packet Length Max        54593 non-null  int64  
 7    Fwd Packet Length Min        54593 non-null  int64  
 8    Fwd Packet Length Mean       54593 non-null  float64
 9    Fwd Packet Length Std        54593 non-null  float64
 10  Bwd Packet Length Max         54593 non-null  int64  
 11   Bwd Packet Length Min        54593 non-null  int64  
 12   Bwd Packet Length Mean       54593 non-null  float64
 13   

In [5]:
# display our datas empty values
data.isna().sum().sum()

np.int64(150)

In [6]:
# removing samples with missing values
data = data.dropna()
data.isna().sum().sum()

np.int64(0)

In [7]:
# we also want to make clear what our labels are so we will assign attack as the new name
data = data.rename(columns={" Label": "attack"})
data.columns[-1]

'attack'

In [8]:
# viewing the different type of attacks
data['attack'].unique()

array(['BENIGN', 'DoS Hulk', 'DoS GoldenEye'], dtype=object)

In [9]:
# we group the dos attacks by assigning 1 if we see dos in the name of the attack label
data['attack'] = data['attack'].apply(lambda x: 1 if "dos" in str(x).lower() else 0 )
data['attack'].tail()

54588    1
54589    1
54590    1
54591    1
54592    1
Name: attack, dtype: int64

In [10]:
#calculating the variance for each column
column_variances = data.var()

# remove data which contains no variance
filtered_data = data.loc[:, column_variances != 0].copy()
for x, col in enumerate(filtered_data.columns):
    print(f"{x:<{2}} {col:<{30}} var : {filtered_data[col].var():>{25}.5f} mean : {filtered_data[col].mean():>{20}.5f}")

0   Destination Port              var :            41914517.35844 mean :            935.35778
1   Flow Duration                 var :    1830912789519646.50000 mean :       40320983.68206
2   Total Fwd Packets             var :                 795.60425 mean :              5.80492
3   Total Backward Packets        var :                1983.40707 mean :              4.48395
4  Total Length of Fwd Packets    var :             1064269.43731 mean :            368.40567
5   Total Length of Bwd Packets   var :          5878887504.61399 mean :           7608.74043
6   Fwd Packet Length Max         var :               92521.46433 mean :            265.69136
7   Fwd Packet Length Min         var :                 226.87373 mean :              2.11589
8   Fwd Packet Length Mean        var :                6115.42847 mean :             52.74535
9   Fwd Packet Length Std         var :               15095.76915 mean :            107.49852
10 Bwd Packet Length Max          var :            10902763.

In [11]:
# displaying some information about the data which has been removed to check if there is any information we should reconsider
removed_data = data.loc[:, column_variances == 0 ]
removed_data

,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,CWE Flag Count,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
54588,0,0,0,0,0,0,0,0,0,0
54589,0,0,0,0,0,0,0,0,0,0
54590,0,0,0,0,0,0,0,0,0,0
54591,0,0,0,0,0,0,0,0,0,0


In [12]:
# viewing if our data is standardized (mean = 0, std =1 )
filtered_data.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,attack
count,54475.000000,5.447500e+04,54475.000000,54475.000000,54475.000000,5.447500e+04,54475.000000,54475.000000,54475.000000,54475.000000,...,54475.000000,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,5.447500e+04,54475.000000
mean,935.357779,4.032098e+07,5.804920,4.483947,368.405672,7.608740e+03,265.691363,2.115888,52.745352,107.498522,...,29.047306,8.619742e+04,1.377269e+04,1.008245e+05,7.705709e+04,3.743201e+07,4.631730e+05,3.776827e+07,3.709637e+07,0.908123
std,6474.142210,4.278917e+07,28.206458,44.535459,1031.634353,7.667390e+04,304.173412,15.062328,78.201205,122.864841,...,5.217411,5.805961e+05,2.332652e+05,6.785944e+05,5.492533e+05,4.305943e+07,5.066017e+06,4.334249e+07,4.307849e+07,0.288855
min,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,80.000000,1.472035e+05,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
50%,80.000000,1.180912e+07,6.000000,5.000000,340.000000,1.159500e+04,329.000000,0.000000,47.571429,126.019707,...,32.000000,6.000000e+00,0.000000e+00,6.000000e+00,6.000000e+00,6.801986e+06,0.000000e+00,6.801986e+06,6.759334e+06,1.000000
75%,80.000000,8.582139e+07,8.000000,6.000000,396.000000,1.159500e+04,374.000000,0.000000,66.500000,156.624152,...,32.000000,1.040000e+03,0.000000e+00,1.040000e+03,1.033000e+03,8.530000e+07,0.000000e+00,8.540000e+07,8.530000e+07,1.000000
max,63849.000000,1.199990e+08,5117.000000,8350.000000,88632.000000,1.450000e+07,11595.000000,1983.000000,2319.000000,5185.441640,...,48.000000,1.370000e+07,6.905038e+06,1.920000e+07,1.370000e+07,1.200000e+08,6.070000e+07,1.200000e+08,1.200000e+08,1.000000


In [13]:
import pandas as np

inf_cols = [' Bwd Packet Length Std','Flow Bytes/s']
filtered_data = filtered_data.drop(columns=inf_cols)
filtered_data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,attack
0,36102,23,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
1,53,30805,2,2,108,230,54,54,54.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,443,5535509,8,8,372,3876,191,0,46.500000,71.876879,...,20,346981.0,0.0,346981,346981,5188524.0,0.0,5188524,5188524,0
3,53,49235,4,2,152,202,38,38,38.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
4,53,181,2,2,70,218,35,35,35.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54588,80,11512204,8,5,326,11632,326,0,40.750000,115.258405,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,1
54589,80,11513325,5,5,471,3525,471,0,94.200000,210.637604,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,1
54590,80,11509201,7,6,314,11632,314,0,44.857143,118.680845,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,1
54591,80,11509095,8,5,369,11632,369,0,46.125000,130.461201,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,1


In [14]:
filtered_data.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' U

In [ ]:
# removing sample with inf values
df = filtered_data.replace(["inf"], "nan")
df = df.dropna()
df.isna().sum().sum()

np.int64(0)

In [24]:
# destination port converted to categories
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df[' Destination Port'] = encoder.fit_transform(df[' Destination Port'])

In [25]:
# x will contain the information outside of the labels for each sample
x = df.loc[:,df.columns != 'attack']

In [26]:
# y will hold only the label values which are stored as true false or 0,1 where the label is considered an attack
y = df['attack'].copy()

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.3,random_state=42,stratify=y)

In [28]:
from sklearn.preprocessing import StandardScaler
#creating a standard scaler to standardize the filtered data
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
# displaying if x train is standardized (mean = 0, std = 1)
pd.DataFrame(X_train).describe()

In [ ]:
# displayed if x test is standardized (mean = 0, std = 1)
pd.DataFrame(X_test).describe()

In [ ]:
(pd.DataFrame(filtered_data).corr().style.background_gradient(cmap='PRGn',vmin=-1,vmax=1))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
np.unique(y_test,return_counts=True)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test,predictions)

print("model accuracy : ", accuracy)

In [ ]:
from xgboost import XGBClassifier

xg_model = XGBClassifier(n_estimators=150).fit(X_train, y_train, verbose = True)
xg_pred = xg_model.predict(X_test)

xg_model

In [ ]:
xg_pred

In [ ]:
# compute the measures
acc_xgboost1 = accuracy_score(y_test, xg_pred)
f1score_xgboost1 = f1_score(y_test, xg_pred)
precision_xgboost1 = precision_score(y_test, xg_pred)
recall_xgboost1 = recall_score(y_test, xg_pred)

# display the measures

print('-'*25)
print('XGBoost Model: 66 features')
print('-'*25)
print('Accuracy: {:.3f}'.format(acc_xgboost1))
print('F1-score: {:.3f}'.format(f1score_xgboost1))
print('Precision: {:.3f}'.format(precision_xgboost1))
print('Recall: {:.3f}'.format(recall_xgboost1))

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, xg_pred)

print("Confusion Matrix:")
print("-"*18)
print(conf_matrix)
print("-"*18)
print("tn,fp")
print("fn,tp")